
# Hyperparameter Sweeps

In this project, we use Hyperparemter sweeps with Pytorch on "Weights & Biases". For further details, check out this [Colab](http://wandb.me/sweeps-colab).

Inspired by https://github.com/SheezaShabbir/Time-series-Analysis-using-LSTM-RNN-and-GRU

## Setup

Start out by installing the experiment tracking library and setting up your free W&B account:

1. Install with `!pip install`
2. `import` the library into Python
3. `.login()` so you can log metrics to your projects

If you've never used Weights & Biases before,
the call to `login` will give you a link to sign up for an account.
W&B is free to use for personal and academic projects!

In [15]:
!pip install wandb -Uq


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [16]:
import wandb

In [17]:
wandb.login()

True

## Defining the sweep config

We define the sweep config via dict in our Jupyter notebook. You can find more information on sweeps in the [documentation](https://docs.wandb.com/sweeps/configuration).

You can find a list of all configuration options [here](https://docs.wandb.com/library/sweeps/configuration) and a big collection of examples in YAML format [here](https://github.com/wandb/examples/tree/master/examples/keras/keras-cnn-fashion).

In [18]:
# See also https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046

sweep_config = {
    'method': 'bayes',
    'metric': {
        'goal': 'minimize',
        'name': 'loss'
    },
    'parameters': {
        'model': {
            'value': 'lstm'
        },
        'batch_size': {
            'distribution': 'q_log_uniform_values',
            'max': 256,
            'min': 32,
            'q': 8
        },
        'train_val_ratio': {
            'value': 0.8
        },
        'dropout': {
            'values': [0, 0.1, 0.2, 0.5]
        },
        'num_layers': {
            'values': [2, 4, 8, 16, 32]
        },
        'epochs': {
            'value': [5, 10, 20, 40]
        },
        'fc_layer_size': {
            'values': [50, 100, 200, 400, 1000]
        },
        'learning_rate': {
            'distribution': 'uniform',
            'max': 0.1,
            'min': 0.00001
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        }
    }
}

## Initialize the setup

In [19]:
sweep_id = wandb.sweep(sweep_config, project="pp-sg-lstm")

Create sweep with ID: haw2fsrn
Sweep URL: https://wandb.ai/parcaster/pp-sg-lstm/sweeps/haw2fsrn


## Run the sweep agent

### Define Your Training Procedure

Before we can actually execute the sweep, we need to define the training procedure that uses those values.

In the functions below, we define a simple fully-connected neural network in PyTorch, and add the following `wandb` tools to log model metrics, visualize performance and output and track our experiments:
* [**`wandb.init()`**](https://docs.wandb.com/library/init) – Initialize a new W&B Run. Each Run is a single execution of the training function.
* [**`wandb.config`**](https://docs.wandb.com/library/config) – Save all your hyperparameters in a configuration object so they can be logged. Read more about how to use `wandb.config` [here](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-config/Configs_in_W%26B.ipynb).
* [**`wandb.log()`**](https://docs.wandb.com/library/log) – log model behavior to W&B. Here, we just log the performance; see [this Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-log/Log_(Almost)_Anything_with_W%26B_Media.ipynb) for all the other rich media that can be logged with `wandb.log`.

For more details on instrumenting W&B with PyTorch, see [this Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Simple_PyTorch_Integration.ipynb).

In [20]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from models import LSTMModel, RNNModel, GRUModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Training on {device}")


def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        X, y, input_dim, output_dim = load_features_labels()
        X_train, X_val, y_train, y_val = split_train_val(X, y, config.train_val_ratio)
        train_loader = build_dataset(config.batch_size, X_train, y_train)
        val_loader = build_dataset(config.batch_size, X_val, y_val)
        network = build_network(config.fc_layer_size, config.dropout, config.num_layers, input_dim, output_dim,
                                config.model)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, train_loader, optimizer, config.batch_size, input_dim)
            avg_val_loss = val_epoch(network, val_loader, config.batch_size, input_dim)
            wandb.log({"loss": avg_loss, "epoch": epoch})
            wandb.log({"loss (validation)": avg_val_loss, "epoch": epoch})

Training on cuda


In [21]:
def load_features_labels():
    df = pd.read_csv("../data/preprocessing/02_pp_sg_train_features.csv", sep=";")

    parking_data_labels = ["P24", "P44", "P42", "P33", "P23", "P25", "P21", "P31", "P53", "P32", "P22", "P52", "P51",
                           "P43"]  # TODO get these from metadata file
    ignored_columns = ["datetime", "date", "year", "month", "day", "weekdayname", "weekday", "time", "hour", "minute"]

    y = df[parking_data_labels]
    X = df.drop(columns=parking_data_labels)
    X = X.drop(columns=ignored_columns)

    input_dim = len(X.columns)
    output_dim = len(y.columns)

    print(f"Input dimension: {input_dim}, columns: {X.columns}")
    print(f"Output dimension: {output_dim}, columns: {y.columns}")

    return X, y, input_dim, output_dim


def split_train_val(X, y, train_val_ratio):
    train_size = int(len(X) * train_val_ratio)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    return X_train, X_val, y_train, y_val

This cell defines the four pieces of our training procedure:
`build_dataset`, `build_network`, `build_optimizer`, and `train_epoch`.

All of these are a standard part of a basic PyTorch pipeline,
and their implementation is unaffected by the use of W&B,
so we won't comment on them.

In [22]:
def build_dataset(batch_size, X, y):
    features = torch.Tensor(X.values)
    targets = torch.Tensor(y.values)

    dataset = TensorDataset(features, targets)

    return DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)


def build_network(fc_layer_size, dropout, num_layers, input_dim, output_dim, model):
    if model == "rnn":
        network = RNNModel(input_dim=input_dim, hidden_dim=fc_layer_size, layer_dim=num_layers, output_dim=output_dim,
                           dropout_prob=dropout)
    elif model == "lstm":
        network = LSTMModel(input_dim=input_dim, hidden_dim=fc_layer_size, layer_dim=num_layers, output_dim=output_dim,
                            dropout_prob=dropout)
    elif model == "gru":
        network = GRUModel(input_dim=input_dim, hidden_dim=fc_layer_size, layer_dim=num_layers, output_dim=output_dim,
                           dropout_prob=dropout)
    else:
        raise ValueError(f"Invalid model value: {model}")

    return network.to(device)


def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer, batch_size, input_dim):
    losses = []
    network.train()
    for _, (data, target) in enumerate(loader):
        data, target = data.view([batch_size, -1, input_dim]).to(device), target.to(device)
        optimizer.zero_grad()

        # output = network(data.unsqueeze(0)).squeeze() # See https://medium.com/@mike.roweprediger/using-pytorch-to-train-an-lstm-forecasting-model-e5a04b6e0e67

        # ➡ Forward pass
        loss = F.mse_loss(network(data), target)
        losses.append(loss.item())

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return np.mean(losses)


def val_epoch(network, loader, batch_size, input_dim):
    losses = []
    with torch.no_grad():
        network.eval()
        for _, (data, target) in enumerate(loader):
            data, target = data.view([batch_size, -1, input_dim]).to(device), target.to(device)
            loss = F.mse_loss(network(data), target)
            losses.append(loss.item())

            wandb.log({"batch loss (validation)": loss.item()})

    return np.mean(losses)

The cell below will launch an `agent` that runs `train` 5 times,
usingly the randomly-generated hyperparameter values returned by the Sweep Controller. Execution takes under 5 minutes.

Now, we're ready to start sweeping! 🧹🧹🧹

Sweep Controllers, like the one we made by running `wandb.sweep`, sit waiting for someone to ask them for a `config` to try out.

That someone is an `agent`, and they are created with `wandb.agent`.
To get going, the agent just needs to know
1. which Sweep it's a part of (`sweep_id`)
2. which function it's supposed to run (here, `train`)
3. (optionally) how many configs to ask the Controller for (`count`)

In [23]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: ebuyuynd with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0
wandb: 	epochs: 20
wandb: 	fc_layer_size: 50
wandb: 	learning_rate: 0.03949398109496424
wandb: 	model: lstm
wandb: 	num_layers: 4
wandb: 	optimizer: sgd
wandb: 	train_val_ratio: 0.8


Input dimension: 18, columns: Index(['ferien', 'feiertag', 'covid_19', 'olma_offa', 'temperature_2m_max',
       'temperature_2m_min', 'rain_sum', 'snowfall_sum', 'sin_minute',
       'cos_minute', 'sin_hour', 'cos_hour', 'sin_weekday', 'cos_weekday',
       'sin_day', 'cos_day', 'sin_month', 'cos_month'],
      dtype='object')
Output dimension: 14, columns: Index(['P24', 'P44', 'P42', 'P33', 'P23', 'P25', 'P21', 'P31', 'P53', 'P32',
       'P22', 'P52', 'P51', 'P43'],
      dtype='object')


batch loss,▁▃▁▃▂▃▁▃▂▃▁▃▂▃▁▃▂▃▁▃▂▃▁▃▂▃▂▃▂▃▂▃▂▃▂▃▂▃▂█
batch loss (validation),▁▄▇▆▁▄▇▆▁▄▇▆▁▃▇▆▁▃▇▆▁▃▇▆▁▃▄▆▁▃▄▆▁▃▄▆▁▃▄█
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,▁███████████████████
loss (validation),█▁▅▅▅▅▅▁█▅▅▅▅▅▁▁▅█▅█
batch loss,5717.71875
batch loss (validation),5734.19092
epoch,19
loss,2682.32468
loss (validation),4614.96757


wandb: Agent Starting Run: oe1vygm9 with config:
wandb: 	batch_size: 168
wandb: 	dropout: 0
wandb: 	epochs: 20
wandb: 	fc_layer_size: 1000
wandb: 	learning_rate: 0.014732909192830004
wandb: 	model: lstm
wandb: 	num_layers: 8
wandb: 	optimizer: sgd
wandb: 	train_val_ratio: 0.8


Input dimension: 18, columns: Index(['ferien', 'feiertag', 'covid_19', 'olma_offa', 'temperature_2m_max',
       'temperature_2m_min', 'rain_sum', 'snowfall_sum', 'sin_minute',
       'cos_minute', 'sin_hour', 'cos_hour', 'sin_weekday', 'cos_weekday',
       'sin_day', 'cos_day', 'sin_month', 'cos_month'],
      dtype='object')
Output dimension: 14, columns: Index(['P24', 'P44', 'P42', 'P33', 'P23', 'P25', 'P21', 'P31', 'P53', 'P32',
       'P22', 'P52', 'P51', 'P43'],
      dtype='object')


...

Input dimension: 18, columns: Index(['ferien', 'feiertag', 'covid_19', 'olma_offa', 'temperature_2m_max',
       'temperature_2m_min', 'rain_sum', 'snowfall_sum', 'sin_minute',
       'cos_minute', 'sin_hour', 'cos_hour', 'sin_weekday', 'cos_weekday',
       'sin_day', 'cos_day', 'sin_month', 'cos_month'],
      dtype='object')
Output dimension: 14, columns: Index(['P24', 'P44', 'P42', 'P33', 'P23', 'P25', 'P21', 'P31', 'P53', 'P32',
       'P22', 'P52', 'P51', 'P43'],
      dtype='object')


batch loss,▃▃▇▁▂▃█▁▂▃█▁▂▄▇▁▁▄▇▁▂▄▇▁▁▄▆▁▁▄▆▁▂▄▅▁▂▄▆▃
batch loss (validation),▄▄▃█▃▃▃▇▃▃▃▇▂▂▃▇▂▂▃▇▂▂▃▇▂▁▃▇▂▁▃▇▂▁▃▇▂▁▃▄
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
loss (validation),█▅▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁
batch loss,2126.37744
batch loss (validation),2951.86377
epoch,19
loss,2606.69894
loss (validation),3008.99218


wandb: Agent Starting Run: e7t4a0lu with config:
wandb: 	batch_size: 144
wandb: 	dropout: 0.1
wandb: 	epochs: 20
wandb: 	fc_layer_size: 50
wandb: 	learning_rate: 0.04503953649927898
wandb: 	model: lstm
wandb: 	num_layers: 8
wandb: 	optimizer: adam
wandb: 	train_val_ratio: 0.8


Input dimension: 18, columns: Index(['ferien', 'feiertag', 'covid_19', 'olma_offa', 'temperature_2m_max',
       'temperature_2m_min', 'rain_sum', 'snowfall_sum', 'sin_minute',
       'cos_minute', 'sin_hour', 'cos_hour', 'sin_weekday', 'cos_weekday',
       'sin_day', 'cos_day', 'sin_month', 'cos_month'],
      dtype='object')
Output dimension: 14, columns: Index(['P24', 'P44', 'P42', 'P33', 'P23', 'P25', 'P21', 'P31', 'P53', 'P32',
       'P22', 'P52', 'P51', 'P43'],
      dtype='object')


batch loss,█▄▃▁▄▁▄▁▃▁▁▁▇▃▂▃▂▁▂▃▃▁▄▁▂▁▂▁▂▂▆▂▂▂▂▁▂▃▂▂
batch loss (validation),▃█▃▂▂▃▁▂▁▃▁▃▁▆▁▅▃▂▃▂▂▃▁▂▁▃▁▃▁▆▁▅▃▂▃▂▂▃▇▂
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss (validation),█▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆
batch loss,2539.21069
batch loss (validation),3699.79639
epoch,19
loss,2650.29095
loss (validation),2759.61093
